In [1]:
import sys
sys.path.append('../../../')

## Template - Bias Mitigation Benchmark ([Holistic AI](https://research.holisticai.com))

**Task:** Binary Classification

**Type:** Postprocessing


This notebook is a template for the Bias Mitigation Benchmark. It can be used to mitigate bias in datasets and models. The notebook is based on the [Holistic AI open source library](https://github.com/holistic-ai/holisticai) and follows the bias mitigation benchmark outlined in [Holistic AI](https://research.holisticai.com).

### Template Structure

The template have the following steps:

1. Setup definition: 
    - select a task: `binary_classification`, `multiclass_classification`, `regression`, `clustering`, `recommender`
    - select a type: `inprocessing`, `preprocessing`, `postprocessing`
2. Mitigator class
    - create a class for you custom mitigator
3. Evaluation
    - evaluate your mitigator and compare it with other mitigators
4. Submission
    - do you have good results? Then submit your mitigator to the Bias Mitigation Benchmark


### Step 1: Setup Definition

In [2]:
from holisticai.benchmark.tasks import task_name, get_task

print(task_name)

['binary_classification', 'multiclass_classification', 'regression', 'clustering', 'recommender']


In [3]:
# load a task
task = get_task("binary_classification")

In [10]:
# benchmark for the task by type
task.benchmark(type='postprocessing')

Dataset,Average AFS,adult,bank_marketing,census_kdd,compas_recidivism,credit_card,diabetes,german_credit,law_school
Mitigator,,,,,,,,,
MLDebiaser,0.891768,0.884651,0.947397,0.960676,0.814375,0.902120,0.804558,0.888181,0.932189
EqualizedOdds,0.879779,0.854772,0.943726,0.960879,0.809373,0.900984,0.800230,0.866197,0.902067
RejectOptionClassification,0.865316,0.880701,0.907784,0.955288,0.821524,0.879057,0.784279,0.765489,0.928405
LPDebiaserBinary,0.847006,0.825449,0.914638,0.905938,0.801134,0.884189,0.790329,0.846687,0.807689
CalibratedEqualizedOdds,0.826822,0.848425,0.942929,0.959960,0.452759,0.887227,0.795279,0.871521,0.856475


### Step 2: Mitigator Class

In [5]:
# additional class for postprocessing mitigation
from holisticai.utils.transformers.bias import SensitiveGroups
from holisticai.bias.mitigation.postprocessing.ml_debiaser.randomized_threshold.algorithm import RandomizedThresholdAlgorithm

# custom postprocessing mitigation
import numpy as np

class MyPostprocessingMitigator():
    """
    My Postprocessing Mitigator
    """

    def fit(self, y_proba, group_a, group_b):
        self.sens_groups = SensitiveGroups()
        sensitive_features = np.stack([group_a, group_b], axis=1)
        
        self.sens_groups.fit(sensitive_features)
        self.algorithm = RandomizedThresholdAlgorithm()
        
        return self

    def transform(self, y_proba, group_a, group_b):
        sensitive_features = np.stack([group_a, group_b], axis=1)
        p_attr = self.sens_groups.transform(sensitive_features, convert_numeric=True)

        pred = y_proba[:, 1]
        pred = (2 * pred - 1) 
        self.algorithm.fit(pred, p_attr)

        new_y_score = self.algorithm.predict(pred, p_attr)
        new_y_pred = np.where(new_y_score > 0.5, 1, 0)

        return {"y_pred": new_y_pred, "y_score": new_y_score}

### Step 3: Evaluation

In [7]:
my_mitigator = MyPostprocessingMitigator()

task.run_benchmark(custom_mitigator = my_mitigator, type = 'postprocessing')

Binary Classification Benchmark initialized for MyPostprocessingMitigator


100%|██████████| 1/1 [00:13<00:00, 13.19s/it]


In [9]:
task.evaluate_table()

Dataset,Average AFS,adult,bank_marketing,census_kdd,compas_recidivism,credit_card,diabetes,german_credit,law_school
Mitigator,,,,,,,,,
MLDebiaser,0.891768,0.884651,0.947397,0.960676,0.814375,0.902120,0.804558,0.888181,0.932189
MyPostprocessingMitigator,0.884738,0.884738,nan,nan,nan,nan,nan,nan,nan
EqualizedOdds,0.879779,0.854772,0.943726,0.960879,0.809373,0.900984,0.800230,0.866197,0.902067
RejectOptionClassification,0.865316,0.880701,0.907784,0.955288,0.821524,0.879057,0.784279,0.765489,0.928405
LPDebiaserBinary,0.847006,0.825449,0.914638,0.905938,0.801134,0.884189,0.790329,0.846687,0.807689
CalibratedEqualizedOdds,0.826822,0.848425,0.942929,0.959960,0.452759,0.887227,0.795279,0.871521,0.856475


### Step 4: Submission

In [ ]:
task.submit()

MyMitigator benchmark submitted
MyMitigator benchmark submitted
https://holistic-ai.com/benchmark/binary_classification
